# Tutorial on DER Hosting Capacity | <font color=red>Part 1</font>: Advanced Tools for the Analysis of Three-Phase Unbalanced LV Networks

## 1. Introduction 

### Objectives 
The objectives of this tutorial are:
1. To familiarise with the process by which power engineers can run a **power flow** for a given **low voltage (LV) distribution network**. To achieve this, you will learn the basics of how to model an LV distribution network and investigate certain aspects such as voltage regulation and energy losses.

2. To continue familiarising with **advanced tools** useful to run distribution network studies involving DERs. You will continue using [OpenDSS](https://www.epri.com/pages/sa/opendss) via the [dss_python](https://github.com/dss-extensions/dss_python) module. And, to guide you, all will be done using a notebook on [Jupyter Notebook](https://jupyter.org/).

### Structure of this Notebook
The rest of this notebook is divided into two parts:

- **2. Tutorial.** Here you will learn, step by step, how to model a distribution network and analyse what-if scenarios by running power flows with OpenDSS and Python.
- **3. Exercises.** Here you will go through some exercises that will help you familiarise with the **`code`** and how to modify it for different purposes.
- **4. Simulation Workspace.** Here you can place all your code to run it at once. You should use **`code`** that is relevant to each exercise.

## 2. Tutorial 

To learn how to use OpenDSS and Python to do LV network analyses, we will first create the simple LV network shown in the figure below. We have a transformer (11kV/0.4kV), a three-phase line connecting bus A with bus B, and three single-phase lines connecting bus B with buses C, D, and E. Buses C, D, and E represent the connection points of three different houses.
* The 11kV and 0.4kV are line-to-line nominal values. Thus, the single-phase houses will have a nominal line-to-neutral value of 230V.

<center> <img style="float: middle;" src="Figure_1.png" width="40%"> </center>

**<center> Figure 1. Simple LV Network</center>** 

Follow the steps below. **And let's begin!**

<font color='red'>**<u>Note</u>:**</font> You will need to run the code in each cell. Select the code cell (a **<font color='blue'>blue</font> or <font color='green'>green</font> box [ ]** surrounding the cell will appear) and then click on the &#9654; **`Run`** button on top of the page to run the corresponding code (or use the shortcut `Control`-`Enter`). Since you will be running code, **you will need to run the cells sequentially**, one after the other without skipping code cells. A number will appear inside the brackets [X] indicating the number of the *run*.

### 2.1 Load Modules
First, we need to run the code below to load the modules that will be used in this tutorial. The purpose of these modules are:

* **os:** Operating system module, it will be used to get the directory path.
* **dss:** Module for interfacing Python code with OpenDSS. Here is more info about [dss_python](https://github.com/dss-extensions/dss_python) and other [DSS Extensions](https://dss-extensions.org/).

<font color='red'>**<u>Note</u>:**</font> If there is an error in this step, that means that some of the modules are not installed in your computer, so go back to the info provided in Part 0 and make sure you have everything installed.

In [ ]:
import os
import dss

<a id='4'></a>
### 2.2 Set the working path
Run the code in the cell below to get your working path. As an output you get the location of this notebook in your computer.  

<font color='red'>**<u>Note</u>:**</font> You should always open the Jupyter Notebook file from the folder that you uploaded to Jupyter Notebook. If you want to continue working later, you can simply save the changes and then open the **.ipynb** file again and continue working from where you left.

In [ ]:
my_directory = os.getcwd()
print(f"The direction is located in the following path: my_directory = {my_directory}")

### 2.3 Set up dss_engine
Before running the code in Python, we need to set up the **dss_engine** which will be used for OpenDSS commands.


In [ ]:
dss_engine = dss.DSS
DSSText = dss_engine.Text
DSSCircuit = dss_engine.ActiveCircuit
DSSSolution = dss_engine.ActiveCircuit.Solution
ControlQueue = dss_engine.ActiveCircuit.CtrlQueue
dss_engine.AllowForms = 0

### 2.4 Network Modelling - Creating a Circuit
Since we have imported all necessary modules and set up the OpenDSS engine, we can move to the actual modelling of the simple LV network presented in Figure 1.

You will notice that we use the **`DSSText.Command = 'command'`** many times. This is how we send commands to OpenDSS, so anything that is after this command and between 'quotes' is writen in the OpenDSS language (e.g., 'Clear').

The first steps to model the network in OpenDSS is to set up a circuit and its main parameters, as explained below.

1. **`Clear`** is used to clear any existent model.
2. **`Set DefaultBaseFrequency=50`** sets the frequency for the circuit (i.e., the network) to 50Hz. (Always set the default base frequency to the correct value. There can be no conflict between this frequency and any other stated frequency in the code.)
3. **`New Circuit.Simple_LV_Network`** creates a new circuit named "Simple_LV_Network". The name put after the point will be the name of your circuit.
4. **`Edit vsource.source bus1=sourceBus basekv=11 pu=1.0 phases=3`** edits the source bus named source (i.e., source bus is how OpenDSS names our old friend slack bus) and set its parameters.

<font color='red'>**<u>Note</u>:**</font> Do not change the order of these commands nor any of the parameters.


In [ ]:
DSSText.Command = 'Clear'
DSSText.Command = 'Set DefaultBaseFrequency=50'
DSSText.Command = 'New Circuit.Simple_LV_Network'
DSSText.Command = 'Edit vsource.source bus1=sourceBus basekv=11 pu=1.0 phases=3'

### 2.5 Adding the 11kV/0.4kV Transformer  
As we can see from Figure 1, the network has **one** transformer. Below is the explanation of the command (next code cell) used to set up the transformer named here as LVTR.
- **`New Transformer.LVTR`** creates a new transformer named LVTR.
- **`Buses=[sourceBus, A.1.2.3]`** sets the two buses that the transformer LVTR is connected to. They are the **sourceBus** and bus **A**.
- **`Conns=[delta wye]`** sets the connection of the three-phase transformer in each bus. The transfomer is connected in a delta-wye configuration.
- **`KVs=[11.0, 0.4]`** sets the transformer ratio, which is 11kV to 0.4kV.
- **`KVAs=[250 250]`** sets the transformer rated capacity in kVA, which is 250kVA.
- **`%Rs=0.00`** sets the series resistance in percentage of pu, which is assumed to be zero.
- **`Xhl=2.5`** sets the reactance in percentage of pu, which is 2.5.
- **`%loadloss=0`** sets the percentage of loss at rated power, which is assumed to be zero.

<font color='red'>**<u>Note</u>:**</font> Although the settings are explained here in a separated way, do not separate them in the code. The code should be as shown below.

In [ ]:
DSSText.Command = 'New transformer.LVTR Buses=[sourcebus, A.1.2.3] Conns=[delta wye] KVs=[11, 0.4] KVAs=[250 250] %Rs=0.00 xhl=2.5 %loadloss=0 '

### 2.6 Creating the linecodes  
When multiple electrical lines have the same characteristics (e.g, conductors with same cross sections), instead of writing those characteristics all over again, we create **linecodes**. We can then use linecodes in the definition of many lines.

Below is the explanation of the command (next code cell) used to set up the **linecode.240sq** which is used to describe a three-phase line segment with conductors with a cross section of 240mm<sup>2</sup>.
- **`New linecode.240sq`** creates a new line code named "240sq".
- **`nphases=3`** sets the number of phases to 3.
- **`R1=0.127`** sets the value of the resistance in ohms per unit length, which is 0.127. 
- **`X1=0.072`** sets the value of the reactance in ohms per unit length, which is 0.072. 
- **`R0=0.342`** sets the value of the zero resistance in ohms per unit length, which is 0.342. 
- **`X0=0.089`** sets the value of the zero reactance in ohms per unit length, which is 0.089. 
- **`Units=km`** sets the units in which their length is expressed, which is kilometers.

The same applies to **linecode.16sq** just with different characteristics.
  
<font color='red'>**<u>Note</u>:**</font> Although the settings are explained here in a separated way, do not separate them in the code. The code should be as shown below.

In [ ]:
DSSText.Command = 'new linecode.240sq nphases=3 R1=0.127 X1=0.072 R0=0.342 X0=0.089 units=km'
DSSText.Command = 'new linecode.16sq nphases=1 R1=1.15 X1=0.083 R0=1.2 X0=0.083 units=km'

#### 2.7 Creating the 400V and 230V lines
We can notice that the LV network consists of **one** three-phase line connecting **bus A and bus B**. This is followed by **three** single-phase lines connecting **B and C**, **B and D**, and **B and E**.

Below is the explanation of the command (next code cell) used to set up the three-phase **LineA_B** and the single-phase **LineB_L1**, **LineB_L2**, and **LineB_L3**.

- Line 1, named "LineA_B" (**`New Line.LineA_B`** in the code), which connects all three phases of bus A (**`bus1=A.1.2.3`** in the code) with all three phases of bus B (**`bus2=B.1.2.3`** in the code). 
- Line 2, named "LineB_L1" (**`New Line.LineB_L1`** in the code), which connects one phase of bus B (**`bus1=B.1`** in the code) with that of bus C (**`bus2=C.1`** in the code). 
- Line 3, named "LineB_L2" (**`New Line.LineB_L2`** in the code), which connects one phase of bus B (**`bus1=B.2`** in the code) with that of bus D (**`bus2=D.1`** in the code). 
- Line 4, named "LineB_L3" (**`New Line.LineB_L3`** in the code), which connects one phase of bus B (**`bus1=B.3`** in the code) with that of bus E (**`bus2=E.1`** in the code).


In [ ]:
DSSText.Command = 'new line.A_B bus1=A.1.2.3 bus2=B.1.2.3 length=1 phases=3 units=km linecode=240sq'
DSSText.Command = 'new line.B_L1 bus1=B.1 bus2=C.1 length=0.01 phases=1 units=km linecode=16sq'
DSSText.Command = 'new line.B_L2 bus1=B.2 bus2=D.1 length=0.01 phases=1 units=km linecode=16sq'
DSSText.Command = 'new line.B_L3 bus1=B.3 bus2=E.1 length=0.01 phases=1 units=km linecode=16sq'

### 2.8 Connecting loads to a bus  
We need to model each single-phase load (each house) according to Figure 1.
* <font color= red>Red house</font> (bus C) will be represented by Load_1 (using <font color= red> phase 1</font>)
* <font color= blue>Blue house</font> (bus D) will be represented by Load_2 (using  <font color= blue> phase 2</font>)
* <font color= limegreen> Green house </font> (bus E) will be represented by Load_3 (using <font color= limegreen> phase 3</font>)  

Below is the explanation of the command (next code cell) used to set up **Load_1**.
- **`New Load.Load_1`** creates a new load named "Load_1".
- **`bus1=C.1`** sets the bus where the load is connected to, which is bus "C" phase 1.
- **`Phases=1`** sets the number of phases for the load, which is single-phase.
- **`kV=(0.4 3 sqrt /)`** which is a Reverse Polish Notation that sets the rated voltage of the single-phase load, which is 230V.
- **`kW=7`** sets the rated active power of the load, which is 7kW.
- **`pf=0.95`** sets the power factor of the load, which is 0.95pu (lagging/inductive).
- **`model=1`** sets the load to constant P+jQ load model.  
- **`conn=wye`** sets the connection type (in this case, wye or star).  
- **`Vminpu=0.85`** sets the minimum voltage in per unit for which the load model is assumed to be valid.
- **`Vmaxpu=1.20`** sets the maximum voltage in per unit for which the load model is assumed to be valid.
- **`Status=Fixed`** defines the variable nature of the load (in this case, fixed).

The same applies to **Load_2** and **Load_3** but considering the parameters in the table below.

| Load Name | Phases | Connected bus | Peak Demand (kW) |  PF  |
|:---------:|:------:|:-------------:|:-----------:|:----:|
|   Load_1  |    1   |       C       |      7      | 0.95 |
|   Load_2  |    1   |       D       |      6      | 0.95 |
|   Load_3  |    1   |       E       |      8      | 0.95 |

<font color='red'>**<u>Note</u>:**</font> Do not change the values of the `model`, `Vminpu` or `Vmaxpu`. For different demands, the values of **`kW`** and **`pf`** should be changed accordingly.

In [ ]:
DSSText.Command = 'new load.Load_1 bus1=C.1 phases=1 kV=(0.4 3 sqrt /) kW=7 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed'
DSSText.Command = 'new load.Load_2 bus1=D.1 phases=1 kV=(0.4 3 sqrt /) kW=6 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed'
DSSText.Command = 'new load.Load_3 bus1=E.1 phases=1 kV=(0.4 3 sqrt /) kW=8 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed'

### 2.9 Set the Control mode and the Voltage bases
Since at this stage we are not controlling network elements (e.g., on-load tap changers) nor we are doing time-varying analyses, we will set our `controlmode` and `mode` to **static** and **snapshot**, respectively. We are just doing a simple, single power flow run.

Here we also set the voltage bases of the circuit (**`Set VoltageBases= [11.0, 0.4]`** in the code).

And with the command **`calcvoltagebases`** we obtain the p.u. values of the voltages. 

In [ ]:
DSSText.Command = 'set controlmode=static'
DSSText.Command = 'set mode=snapshot'
DSSText.Command = 'Set VoltageBases=[11 0.4]'
DSSText.Command = 'calcvoltagebases'

### 2.10 Run the power flow simulation  
We will run the power flow simulation for our circuit using the command **`DSSSolution.Solve()`**.

If the Solution converged, you will see a note saying so (and the results if applicable). If not, then there is a mistake in the code. In that case, usually a <font color='red'>DSSException</font> cell appears bellow the cell that has an error, giving direction where the error is located.

<font color='red'>**<u>Note</u>:**</font> This command comes **after modelling all of the elements of the network**.


In [ ]:
DSSSolution.Solve()
if DSSSolution.Converged:
    print("The Circuit was Successfully Solved :-)")
    print("\n")

### 2.11 Extracting the results
Once we have solved the circuit we can **extract the results** from the successful power flow simulation.

There are different ways and different commands for extracting the results. Some of them are listed below.

#### 2.11.1 Load Active (P) and Reactive (Q) Power
Although, in our case, we have already specified these values, we can verify them.

In order to extract the active and reactive power of the loads, there are two consecutive commands in OpenDSS:
- The first one sets the specific load element as an **`active`** element.
- The second one extracts the **`active power (P) [kW]`** or **`reactive power (Q) [kvar]`** of the load.

Since we are working with **three loads**, we need to set them active one by one and extract their P and Q values.

In [ ]:
active_load = "Load_1" 
DSSCircuit.SetActiveElement(f"Load.{active_load}")
print(f"{active_load}:  ")
print(f"Active power demand (P)=", round(DSSCircuit.ActiveElement.Powers[0], 3), "kW")
print(f"Reactive power demand (Q)=", round(DSSCircuit.ActiveElement.Powers[1], 3), "kvar")
print("\n")

active_load = "Load_2" 
DSSCircuit.SetActiveElement(f"Load.{active_load}")
print(f"{active_load}:  ")
print(f"Active power demand (P)=", round(DSSCircuit.ActiveElement.Powers[0], 3), "kW")
print(f"Reactive power demand (Q)=", round(DSSCircuit.ActiveElement.Powers[1], 3), "kvar")
print("\n")

active_load = "Load_3" 
DSSCircuit.SetActiveElement(f"Load.{active_load}")
print(f"{active_load}:  ")
print(f"Active power demand (P)=", round(DSSCircuit.ActiveElement.Powers[0], 3), "kW")
print(f"Reactive power demand (Q)=", round(DSSCircuit.ActiveElement.Powers[1], 3), "kvar")

#### 2.11.2 Voltage magnitudes of the single-phase buses (connection point of the houses)
To extract the voltages of the load buses, there are 4 consecutive commands in OpenDSS:
- The first one sets the specific load as an **`active`** element.
- The second one extracts the name of the bus to which the load is connected and stores it in a variable called `bus_name`. 
- The third one sets the corresponding bus as an **`active`** element.
- The fourth one extracts the **`voltage magnitude`** of the active bus in p.u.

To **extract the voltage magnitudes of the buses connected to the other loads**, you just need to change the name of the load (similar as done in the previous subsection **2.11.1**)

In [ ]:
active_load = "Load_1" 
DSSCircuit.SetActiveElement(f"Load.{active_load}")
bus_name = DSSCircuit.ActiveElement.Properties('bus1').Val
DSSCircuit.SetActiveBus(bus_name)
print(f"The voltage [in p.u.] of the bus connected to {active_load} =", round(DSSCircuit.ActiveBus.puVmagAngle[0], 3), "pu")

#### 2.11.3 Voltage magnitudes of the three-phase buses 
To extract the voltages of the three-phase buses, there are three consecutive commands in OpenDSS:
- The first one sets the bus we want to extract the voltage magnitudes from (`active_bus = "A"`).
- The second one sets the specific bus as an **`active`** element.
- The third one extracts the **`voltage magnitude`** of the active bus in p.u. per phase.
  - The results of the voltage magnitudes are given in a matrix where the even numbers: 0, 2, and 4 give the voltage magnitudes of phases 1, 2, and 3, respectively. 

In [ ]:
active_bus = "A"  # ← only change the bus name (sourcebus, A, B)
DSSCircuit.SetActiveBus(active_bus)
print(f"Voltage magnitudes at bus {active_bus}:  ")
print(f"Voltage magnitude - phase 1 =", round(DSSCircuit.ActiveBus.puVmagAngle[0], 3), "pu")
print(f"Voltage magnitude - phase 2 =", round(DSSCircuit.ActiveBus.puVmagAngle[2], 3), "pu")
print(f"Voltage magnitude - phase 3 =", round(DSSCircuit.ActiveBus.puVmagAngle[4], 3), "pu")

#### 2.11.4 Active (P) and reactive (Q) power of the transformer
To extract the active and reactive power of the transformer per phase there are two consecutive commands in OpenDSS:  
- The first one sets the transformer as an **`active`** element.
- The second one extracts the P and Q per phase such that:
  - The even numbers: **8, 10, and 12** give the values of the `active power (P) [kW]` of phases 1, 2 and 3, respectively.
  - The odd numbers: **9, 11, and 13** give the values of the `reactive power (Q) [kvar]` of phases 1, 2 and 3, respectively.

In [ ]:
DSSCircuit.SetActiveElement('transformer.LVTR')
print(f"Results of the transformer LVTR: ")
print(f"Active power (P) supplied to phase 1 =", round(abs(DSSCircuit.ActiveElement.Powers[8]), 3), "kW")
print(f"Active power (P) supplied to phase 2 =", round(abs(DSSCircuit.ActiveElement.Powers[10]), 3), "kW")
print(f"Active power (P) supplied to phase 3 =", round(abs(DSSCircuit.ActiveElement.Powers[12]), 3), "kW")
print(f"Rective power (Q) supplied to phase 1 =", round(abs(DSSCircuit.ActiveElement.Powers[9]), 3), "kvar")
print(f"Rective power (Q) supplied to phase 2 =", round(abs(DSSCircuit.ActiveElement.Powers[11]), 3), "kvar")
print(f"Rective power (Q) supplied to phase 3 =", round(abs(DSSCircuit.ActiveElement.Powers[13]), 3), "kvar")

#### 2.11.5 Line Losses
To extract the losses of the lines there are two consecutive commands in OpenDSS:
- The first one stores all the losses in a variable named `"line_loss"`.
- The second one extracts the losses, such that the first value gives the active power losses (in `kW`) and the second value gives the reactive power losses (in `kvar`).

In [ ]:
line_loss = DSSCircuit.LineLosses
print(f"Results of the Power Losses: ")
print(f"Active power (P) losses =", round(abs(line_loss[0]), 3), "kW")
print(f"Reactive power (Q) losses =", round(abs(line_loss[1]), 3), "kvar")

## 3. Exercises

First, read all the exercises so you understand their purpose.

At the very end of this notebook in **4. Simulation Workspace**, you will be able to place all your code to run it at once. Remember, you should use **`code`** that is relevant to each exercise.

### **Exercise 1: Expanding the Simple LV network**

This exercise covers two expansions. In the first one, you need to expand the LV network from Figure 1 to the one shown in Figure 2 (three new houses). In the second one, you will need to add **solar PV systems** to those new houses, as shown in Figure 3.

<center> <img style="float: middle;" src="Figure_2.png" width="50%"> </center>

**<center> Figure 2. Expanded Simple LV Network</center>** 

Include the new elements described below using, as starting point, the code from the previous section (**2. Tutorial**). Remember to use the `relevant cell` for each type of element.

- One **`three-phase line`** connecting bus **B** and bus **F**
- Three **`single-phase lines`** connecting: 
  - **Bus F** (phase 1) and **bus G** : length = 0.01 km, linecode=16sq;
  - **Bus F** (phase 2) and **bus H** : length = 0.01 km, linecode=16sq;
  - **Bus F** (phase 3) and **bus J** : length = 0.01 km, linecode=16sq;
- Three **`single-phase loads`**: 
  - **`Load_4`** connected to **bus G** 
  - **`Load_5`** connected to **bus H** 
  - **`Load_6`** connected to **bus J** 

The load parameters for the three single-phase loads are given in `Table 1`. The characteristics of the annual load duration curve are given in `Table 2`.


<center> Table 1. Load Parameters</center>


 
| Load Name | Phases | Connected bus | Peak Demand (kW) |  PF  |
|:---------:|:------:|:-------------:|:-----------:|:----:|
|   Load_4  |    1   |       G       |      7      | 0.95 |
|   Load_5  |    1   |       H       |      9      | 0.95 |
|   Load_6  |    1   |       J       |      8      | 0.95 |



<center> Table 2. Annual Load Duration</center>

 
|  <center> Peak</center>  |    <center>Medium (67% of Peak)</center>  |  <center>Minimum (33% of Peak)</center>|
| :------:                 | :------:                                  | :------:                               |
|<center>1650 hours</center>|  <center>4280 hours</center>|  <center>2830 hours </center>|


**E1.1:** Calculate the total annual energy losses of the circuit in kWh. Is this value different from the previous case (Figure 1)? If so, why?

**E1.2:** Extract the voltages (in pu) of all the houses for each loading level. Considering that the upper and lower statutory voltage limits are +10% and - 6% of nominal, are those voltages compliant with these limits at all times? If not, why?

Now, you will need to add the **solar PV systems** to each of new houses as shown in Figure 3.

The code to add a PV system is the following: `DSSText.Command = 'new PVSystem.PV4 bus1=G.1 phases = 1 kva = 5 pmpp = 5 pf = 1 kV=(0.4 3 sqrt /) model=1'`, where: 
- **`New PVSystem.PV4`** creates a new PV system named "PV4". (because it is connected to the same bus as Load_4)
- **`bus1=C.1`** sets the bus where the PV system is connected to, which is bus "C" phase 1.
- **`Phases=1`** sets the number of phases for the PV system, which is single-phase.
- **`kva= 5`** is the PV system maximum installed capacity, which is 5kVA.
- **`pmpp=5`** sets the maximum power point of the PV systems, or the maximum output of the PV, which is 5kW.
- **`pf=1`** sets the power factor of the PV system, which in this case is unity (no reactive power).
- **`kV=(0.4 3 sqrt /)`** which is a Reverse Polish Notation that sets the rated voltage of the PV system, which is 230V.
- **`model=1`** sets the PV system to constant P+jQ model (**do not change this value**). 

The same PV system is added to bus H and bus J.  

<font color='red'>**<u>Note</u>:**</font> You should include the code for PV systems right after the code for connecting loads to a bus.

<font color='purple'>**<u>Tech Note</u>:**</font> The kVA is for the inverter and the pmpp is for the panels. These numbers can be different. For instance, you can have a large roof where you can place 8kW of panels but your distribution company imposes an export limit of 5kW. In this case, you would use kVA=5 and pmpp=8.

</br>

<center> <img style="float: middle;" src="Figure_3.png" width="50%"> </center>

**<center> Figure 3. Expanded Simple LV Network with Solar PV Systems</center>** 

**E1.3:** Once you add the PV systems, extract the voltages (in pu) of all the houses for each loading level. Are those voltages compliant with the statutory limits at all times? If not, why?

### **Exercise 2: Modelling a New Simple LV Network**

In this exercise, you will model a new LV network from scratch.

<font color='red'>**<u>Note</u>:**</font> You must follow the sequence as in **"2. Tutorial"**, so that your power flow converges.

The configuration of the LV network is given in Figure 4. 

<center> <img style="float: middle;" src="Figure_4.png" width="60%"> </center>

**<center> Figure 4. New Simple LV Network</center>** 

Here are the characteristics of the elements:
- **`SourceBus`**: 11kV, three-phase 
- **`Transformer`**: delta/wye connection, 11kV to 0.4kV TR ratio, 250kVA rated capacity, 0% series resistance, 2.5pu reactance, 0% percentage of loss at rated power.
- **`Linecodes`**:
  - **Linecode.240sq**: 3-phase, resistance=0.127, reactance=0.072, zero resistance=0.342, zero reactance=0.089 , units=km
  - **Linecode.16sq**: 1-phase, resistance=1.15, reactance=0.083, zero resistance=1.2, zero reactance=0.083 , units=km 
  - **Linecode.185sq**: 3-phase, resistance=0.78, reactance=0.042, zero resistance=1.08, zero reactance=0.093 , units=km
- **`3-phase Lines`**: 
  - **Line.A_B_1**: length=1km, linecode=240sqmm
  - **Line.A_B_2**: length=1km, linecode=240sqmm
  - **Line.B_C**: length=0.8km, linecode=240sqmm
  - **Line.C_D**: length=0.6km, linecode=240sqmm
  - **Line.D_J**: length=0.01km, linecode=185sqmm
- **`Single-phase lines`**: 
  - **Line.B_E**: length=0.01km, linecode=16sqmm
  - **Line.C_F**: length=0.01km, linecode=16sqmm
  - **Line.C_G**: length=0.01km, linecode=16sqmm
  - **Line.D_H**: length=0.01km, linecode=16sqmm
- **`Single-phase Loads`**: 
  - **Load_1**: connected to bus E, rated voltage: (0.4 3 sqrt /), rated active power and power factor (Table 3), constant P+jQ load model, Vminpu=0.85, Vmaxpu=1.20
  - **Load_2**: connected to bus F, rated voltage: (0.4 3 sqrt /), rated active power and power factor (Table 3), constant P+jQ load model, Vminpu=0.85, Vmaxpu=1.20
  - **Load_3**: connected to bus G, rated voltage: (0.4 3 sqrt /), rated active power and power factor (Table 3), constant P+jQ load model, Vminpu=0.85, Vmaxpu=1.20
  - **Load_4**: connected to bus H, rated voltage: (0.4 3 sqrt /), rated active power and power factor (Table 3), constant P+jQ load model, Vminpu=0.85, Vmaxpu=1.20
- **`Three-phase Loads`**: 
  - **Load_5**: connected to bus J, rated voltage: 0.4, rated active power and power factor (Table 3), constant P+jQ load model, Vminpu=0.85, Vmaxpu=1.20
  
 - **`PV Systems`**: **Include them only when it is specified in the exercise**
   - **PV_1**: connected to bus E, single-phase, total installed capacity= 5kVA, maximum power point =5kW, rated voltage: (0.4 3 sqrt /)
   - **PV_2**: connected to bus F, single-phase, total installed capacity= 5kVA, maximum power point =5kW, rated voltage: (0.4 3 sqrt /)
   - **PV_3**: connected to bus G, single-phase, total installed capacity= 5kVA, maximum power point =5kW, rated voltage: (0.4 3 sqrt /)
   - **PV_4**: connected to bus H, single-phase, total installed capacity= 5kVA, maximum power point =5kW, rated voltage: (0.4 3 sqrt /)
  
The load parameters for the loads are given in `Table 3`, whereas the characteristics of the annual load duration curve are given in `Table 4`.

<br>
 <center> Table 3. Load Parameters </center>

| Load Name | Phases | Connected bus | Demand (kW) |  PF  |
|:---------:|:------:|:-------------:|:-----------:|:----:|
|   Load_1  |    1   |       E       |      6      | 0.95 |
|   Load_2  |    1   |       F       |      4      | 0.95 |
|   Load_3  |    1   |       G       |      6      | 0.95 |
|   Load_4  |    1   |       H       |      7      | 0.95 |
|   Load_5  |    3   |       J       |     15      | 0.95 |

<br>
<center> Table 4. Annual Load Duration</center>

 
|  <center> Peak</center>  |    <center>Medium (75% of Peak)</center>  |  <center>Minimum (25% of Peak)</center>|
| :------:                 | :------:                                  | :------:                               |
|<center>1750 hours</center>|  <center>4380 hours</center>|  <center>2630 hours </center>|

For the network configuration given in `Figure 4`:

**E2.1:** Calculate the total annual energy losses of the circuit in kWh. 

**E2.2:** Extract the voltages (in pu) of all the houses for each loading level. Are those voltages compliant with the statutory limits at all times? If not, why?

Now, consider the network configuration given in `Figure 5` and add the corresponding solar PV systems.

<center> <img style="float: middle;" src="Figure_5.png" width="60%"> </center>

**<center> Figure 5. New Simple LV Network with PV Systems </center>** 

**E2.3:** Calculate the total annual energy losses of the circuit in kWh. Is this value different from the previous case (Figure 4)? If so, why?

**E2.4:** Extract the voltages (in pu) of all the houses for each loading level. Are those voltages compliant with the statutory limits at all times? If not, why?

## 4. Simulation Workspace